# Analysis Scripts for Transmissivity

In [1]:
# %% Imports
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.integrate
import scipy.interpolate

## Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm import tqdm

## Utility libraries
from netCDF4 import Dataset
import glob

from c1lgkt.fields.equilibrium import Equilibrium
from c1lgkt.fields.field_handlers import GaussHermiteFieldHandler, XgcZonalFieldHandler, GaussHermiteFunction
from c1lgkt.fields.field_interpolators import compute_balloon_interpolation
from c1lgkt.fields.geometry_handlers import XgcGeomHandler

import c1lgkt.particles.particle_motion as particle_motion
import c1lgkt.particles.particle_tools as particle_tools

In [2]:
# %% Load files and perform setup

eq = Equilibrium.from_eqdfile(R'D:\Documents\IFS\hmode_jet\D3D141451.eqd')

xgcdata = Dataset(R'D:\Documents\Globus\XGC1.nc')

geom_files = {
    'ele_filename': R'D:\Documents\IFS\hmode_jet\Seo.eqd.ele',
    'fdmat_filename': R'D:\Documents\IFS\hmode_jet\fdmat.pkl',
    'min_e_filename': R'D:\Documents\IFS\hmode_jet\min_E_mat.pkl'
}
geom = XgcGeomHandler(eq, xgcdata, **geom_files)

uph = np.load('./outputs/phase_vel.npz')['u_lstsq']

# Set up zonal interpolation function
tind = 400
ksurf0 = 196
zpot = xgcdata['pot00'][tind,:]
zpot_psi = xgcdata['psi00'][:]
interp_zpot = scipy.interpolate.CubicSpline(zpot_psi, zpot, extrapolate=True)
zonalFields = XgcZonalFieldHandler(eq, xgcdata, tind)

# Set up ballooning mode interpolator
fit_results = np.load('./outputs/fit_results.npz', allow_pickle=True)
params_g, params_gh = fit_results['params_g'], fit_results['params_gh']

# Set up the interpolator
mode = GaussHermiteFunction(params_g[:4], params_gh)
interp_balloon = [(39, mode)]

ballFields = GaussHermiteFieldHandler(geom, interp_zpot, interp_balloon)

tind0 = tind #424, 386

omega_frame = -uph[tind0,ksurf0]*geom.q_surf[ksurf0]*1e-3
rotating_frame = particle_motion.RotatingFrameInfo(0, omega_frame, tind0)
t0 = rotating_frame.t0

In [34]:
# %% Load data and compute Poincare sections

filelabel = 'transmissivity_analysis_test'
output_dir = 'D:/Documents/IFS/hmode_jet/outputs/'

paths = glob.glob(output_dir+'sections/{}/[0-9][0-9][0-9][0-9][0-9].npz'.format(filelabel))

num_saves = len(paths)
print('num_saves: {}'.format(num_saves))
ncheckpoint = 800

t = np.empty(num_saves*ncheckpoint)
y = None
dy = None

# TODO: Refactor so we don't load the entire trajectory into memory to compute the punctures
for kc in range(num_saves):
    with np.load(output_dir + 'sections/{}/{:05d}.npz'.format(filelabel, kc)) as data:
        t[kc*ncheckpoint:(kc+1)*ncheckpoint] = data['t']
        if kc == 0:
            y = np.empty((data['y'].shape[0], len(t)))
            dy = np.empty((data['y'].shape[0], len(t)))
            
        y[:,kc*ncheckpoint:(kc+1)*ncheckpoint] = data['y']
        dy[:,kc*ncheckpoint:(kc+1)*ncheckpoint] = data['dy']

nump = y.shape[0]//5

# Compute Poincare section
#ppunc, npunc = particle_tools.compute_midplane_punctures(t, y, geom)
#ppunc2, npunc2 = particle_tools.compute_toroidal_punctures(t, y, rotating_frame, period=(2*np.pi/39), offset = (-np.pi/39))


num_saves: 20


In [35]:
# %% Compute angular momenta

pp = particle_motion.deut

ham, lphi = particle_tools.compute_integrals_dk(t, y, eq, pp, zonalFields, rotating_frame)

In [36]:
# %% Compute toroidal winding numbers

# Initial parallel velocity
vll0 = y[3*nump:4*nump,0]
# Maximum jump in toroidal canonical angular momentum (i.e. radius)
dlphi = np.max(lphi, axis=1) - lphi[:,0]

# Geometric poloidal angle around the magnetic axis
theta = np.unwrap(np.arctan2(y[0*nump:1*nump,:]-eq.raxis, y[2*nump:3*nump,:]-eq.zaxis), axis=1)

vll_mean = np.mean(vll0)

# Bounce poloidal angle
btheta = np.unwrap(np.arctan2((y[3*nump:4*nump,:]-vll_mean)/pp.vt, y[2*nump:3*nump,:]-eq.zaxis), axis=1)

## Use the Birkhoff weighted average to compute rotation numbers

# Discrete time s
s = np.linspace(0,1, theta.shape[1]-1, endpoint=False)
# weight function
w = np.zeros(theta.shape[1]-1)
w[1:] = np.exp(-1.0 / (s[1:] * (1-s[1:])))

dtheta = np.sum(w[np.newaxis,:] * np.diff(theta, axis=1), axis=1) / np.sum(2*np.pi*w)
dtor = np.sum(w[np.newaxis,:] * np.diff(y[1*nump:2*nump,:], axis=1), axis=1) / np.sum(2*np.pi*w)
dbtheta = np.sum(w[np.newaxis,:] * np.diff(btheta, axis=1), axis=1) / np.sum(2*np.pi*w)

lphi_std = np.std(lphi, axis=1)

In [37]:
%matplotlib qt
plt.figure()
for k in range(lphi.shape[0]):
    color = 'tab:orange' if np.abs(dbtheta[k]) > np.abs(dtheta[k]) else 'tab:blue'
    plt.plot(lphi[k,:]/eq.psix, c=color, alpha=0.05)

In [38]:
%matplotlib qt
plt.figure()
plt.scatter(vll0/pp.vt, dlphi/eq.psix, alpha=0.05)

In [39]:
vll0_grid = vll0.reshape((48,-1)).T
dlphi_grid = dlphi.reshape((48,-1)).T

plt.figure()
plt.plot(vll0_grid[:,0]/pp.vt, np.sum(dlphi_grid[:,:]/eq.psix > 0.04, axis=1)/48, c='tab:blue', marker='.')

In [17]:
vll_mean/pp.vt

0.006044937284776164

In [32]:
plt.figure()

for k in range(nump):
    plt.plot(btheta[k,:])

In [48]:
plt.figure()
k = 0
plt.plot(y[0*nump+k,:], y[2*nump+k,:])

In [13]:
vll0

array([-965.08084491, -836.22700978, -707.37317465, -578.51933953,
       -449.6655044 , -320.81166927, -191.95783414,  -63.10399901,
         65.74983611,  194.60367124,  323.45750637,  452.3113415 ,
        581.16517663,  710.01901176,  838.87284688,  967.72668201,
       -965.08084491, -836.22700978, -707.37317465, -578.51933953,
       -449.6655044 , -320.81166927, -191.95783414,  -63.10399901,
         65.74983611,  194.60367124,  323.45750637,  452.3113415 ,
        581.16517663,  710.01901176,  838.87284688,  967.72668201,
       -965.08084491, -836.22700978, -707.37317465, -578.51933953,
       -449.6655044 , -320.81166927, -191.95783414,  -63.10399901,
         65.74983611,  194.60367124,  323.45750637,  452.3113415 ,
        581.16517663,  710.01901176,  838.87284688,  967.72668201,
       -965.08084491, -836.22700978, -707.37317465, -578.51933953,
       -449.6655044 , -320.81166927, -191.95783414,  -63.10399901,
         65.74983611,  194.60367124,  323.45750637,  452.31134

In [24]:
dbtheta

array([ 1.12583148e-11, -6.71043435e-11, -4.29261404e-10, -3.23393889e-09,
        3.25911649e-11, -7.41408669e-08,  3.97580915e-07, -7.83518998e-04,
       -7.89982790e-04,  2.99437883e-08,  2.46557744e-08,  1.39756261e-08,
        1.65506787e-10,  1.31586057e-09, -7.83611857e-11,  3.18017449e-12,
       -3.48548622e-11, -1.31621584e-11,  1.26749217e-11,  1.31602312e-08,
       -7.74687086e-12, -3.49765914e-08, -3.49801122e-07, -7.82930381e-04,
       -7.89043332e-04,  4.53070643e-08, -2.74883031e-08, -5.12539532e-08,
        1.60073293e-08, -4.19011822e-09,  3.21830425e-10,  4.15970933e-12,
       -2.07782306e-12, -1.91580618e-12, -2.17266469e-12,  1.78403700e-09,
       -1.91612788e-11,  7.08907440e-08, -8.94505690e-08, -7.82987185e-04,
       -7.89068554e-04, -7.03322792e-08, -4.88516791e-08, -2.86282454e-08,
        1.69275189e-08,  2.37191037e-09, -2.68098159e-10, -1.05233174e-11,
        4.71717123e-11,  5.79054739e-12,  1.81166767e-11,  4.64354525e-12,
       -1.97661819e-07,  

In [42]:
plt.figure()
plt.plot(np.polynomial.hermite_e.hermegauss(64)[0], marker='.')